In [43]:
import pandas as pd
df = pd.read_csv("logs_10_20_0_105.csv")
unique_domains = df['rrname'].dropna().str.lower().unique()
pd.DataFrame(unique_domains, columns=['rrname']).to_csv('unique_domains_105.csv', index=False)


/tmp/ipython-input-2131121166.py:2: DtypeWarning: Columns (6,7,8,9,16,17,18,19,20,21,23) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("logs_10_20_0_105.csv")


In [45]:
import pandas as pd
import re

# === 1️⃣ Завантажуємо CSV ===
df = pd.read_csv('unique_domains_105.csv')  # <-- твій файл
if 'category' not in df.columns:
    df['category'] = 'Other'  # якщо колонки ще не було


# 1️⃣ Витягуємо всі домени, які поки позначені як "Other"
unknowns = df[df['category'] == 'Other']['rrname'].unique()

# 2️⃣ Зберігаємо їх у окремий CSV для перевірки
pd.DataFrame(unknowns, columns=['rrname']).to_csv('unlabeled_domains_105.csv', index=False)

# 3️⃣ Завантажуємо цей файл
df_unlabeled = pd.read_csv('unlabeled_domains_105.csv')

# 4️⃣ Визначаємо правила для напівавтоматичного категоризування
rules = {
    r'finance\d*\.example': 'Finance',
    r'client\d*\.sales\.example': 'Sales',
    r'ds\d*\.lab\.example': 'Research/Lab',
    r'devops\d*\.internal': 'IT/Infrastructure',
    r'hr\d*\.example': 'HR',
    r'marketing\d*\.example': 'Marketing',
    r'test\d*\.example': 'Testing',
    'teams': 'Teams',
    'skype': 'Teams',
    'microsoft': 'Cloud/Office',
    'outlook': 'Email',
    'office': 'Email',
    'gmail': 'Email',
    'google': 'Search',
    'gstatic': 'Google Services',
    'github': 'Dev',
    'gitlab': 'Dev',
    'bitbucket': 'Dev',
    'pypi': 'Dev',
    'python': 'Dev',
    'zoom': 'Meetings',
    'meet': 'Meetings',
    'slack': 'Communication',
    'discord': 'Communication',
    'linkedin': 'Social',
    'facebook': 'Social',
    'instagram': 'Social',
    'twitter': 'Social',
    'tiktok': 'Social',
    'youtube': 'Video',
    'netflix': 'Video',
    'vimeo': 'Video',
    'cdn': 'CDN',
    'cloudflare': 'CDN',
    'akamai': 'CDN',
    'aws': 'Cloud',
    'azure': 'Cloud',
    'gcp': 'Cloud',
    'dropbox': 'Storage',
    'onedrive': 'Storage',
    'ads': 'Ads',
    'doubleclick': 'Ads',
    'googletag': 'Ads',
    'analytics': 'Analytics',
}



# 5️⃣ Функція для визначення категорії
def categorize(domain):
    for pattern, category in rules.items():
        if re.search(pattern, domain, re.IGNORECASE):
            return category
    return 'Other'

# 6️⃣ Застосовуємо автолейблінг
df_unlabeled['category'] = df_unlabeled['rrname'].apply(categorize)

# 7️⃣ Зберігаємо результат для ручної перевірки
df_unlabeled.to_csv('unlabeled_domains_labeled_105.csv', index=False)

print("✅ Збережено файл 'unlabeled_domains_labeled_105.csv'. Перевір і за потреби виправ категорії вручну.")


✅ Збережено файл 'unlabeled_domains_labeled_105.csv'. Перевір і за потреби виправ категорії вручну.


In [48]:

# Завантаж свої логі
df = pd.read_csv('logs_10_20_0_105.csv', parse_dates=['timestamp'])

labels = pd.read_csv('unlabeled_domains_labeled_105.csv')
df = df.merge(labels, on='rrname', how='left')

df['category'] = df['category'].fillna('Other')

df['hour'] = df['timestamp'].dt.hour
df['dayofweek'] = df['timestamp'].dt.dayofweek

user_sequences = (
    df.groupby('src_ip')
      .apply(lambda x: x.sort_values('timestamp')['category'].tolist())
)


/tmp/ipython-input-1959171129.py:2: DtypeWarning: Columns (6,7,8,9,16,17,18,19,20,21,23) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('logs_10_20_0_105.csv', parse_dates=['timestamp'])
/tmp/ipython-input-1959171129.py:14: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.sort_values('timestamp')['category'].tolist())


In [54]:
# Припустимо, у тебе є одна велика послідовність категорій
seq = user_sequences[0]

window_size = 20  # довжина історії, наприклад 20
samples = []

for i in range(window_size, len(seq)):
    samples.append(seq[i - window_size:i + 1])  # 20 попередніх + наступна

print(f"Згенеровано {len(samples)} прикладів")

# Перетворюємо в X, y
encoded_sequences = [encoder.transform(s) for s in samples]
X = pad_sequences([s[:-1] for s in encoded_sequences], maxlen=window_size)
y = [s[-1] for s in encoded_sequences]
y = to_categorical(y, num_classes=len(encoder.classes_))

print(X.shape, y.shape)


Згенеровано 416782 прикладів
(416782, 20) (416782, 4)


In [55]:
from keras.callbacks import EarlyStopping

early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

history = model.fit(
    X, y,
    epochs=50,
    batch_size=128,
    validation_split=0.1,
    callbacks=[early_stop],
    verbose=1
)


Epoch 1/50
2931/2931 ━━━━━━━━━━━━━━━━━━━━ 25s 8ms/step - accuracy: 0.9017 - loss: 0.3154 - val_accuracy: 0.9635 - val_loss: 0.1895
Epoch 2/50
2931/2931 ━━━━━━━━━━━━━━━━━━━━ 24s 8ms/step - accuracy: 0.9370 - loss: 0.2013 - val_accuracy: 0.9616 - val_loss: 0.1922
Epoch 3/50
2931/2931 ━━━━━━━━━━━━━━━━━━━━ 40s 8ms/step - accuracy: 0.9372 - loss: 0.1997 - val_accuracy: 0.9627 - val_loss: 0.1785
Epoch 4/50
2931/2931 ━━━━━━━━━━━━━━━━━━━━ 22s 8ms/step - accuracy: 0.9374 - loss: 0.1982 - val_accuracy: 0.9631 - val_loss: 0.1807
Epoch 5/50
2931/2931 ━━━━━━━━━━━━━━━━━━━━ 25s 8ms/step - accuracy: 0.9376 - loss: 0.1974 - val_accuracy: 0.9635 - val_loss: 0.1802
Epoch 6/50
2931/2931 ━━━━━━━━━━━━━━━━━━━━ 23s 8ms/step - accuracy: 0.9377 - loss: 0.1958 - val_accuracy: 0.9626 - val_loss: 0.1897


In [56]:
# Наприклад, хочемо передбачити для користувача з IP
target_ip = '10.20.0.15'  # підстав свій

# Беремо всі його запити
user_data = df[df['src_ip'] == target_ip].sort_values('timestamp')

# Беремо тільки категорії
seq = user_data['category'].tolist()

# Обмежуємо останні max_len (наприклад, 50)
seq = seq[-max_len:]

seq_encoded = encoder.transform(seq)
seq_padded = pad_sequences([seq_encoded], maxlen=max_len)

pred = model.predict(seq_padded)
pred_idx = np.argmax(pred)
pred_category = encoder.inverse_transform([pred_idx])[0]

print(f"Ймовірна наступна категорія для {target_ip}: {pred_category}")

top3_idx = pred[0].argsort()[-3:][::-1]
top3_cats = encoder.inverse_transform(top3_idx)
top3_probs = pred[0][top3_idx]

for cat, prob in zip(top3_cats, top3_probs):
    print(f"{cat}: {prob:.2%}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 197ms/step
Ймовірна наступна категорія для 10.20.0.15: Other
Other: 92.04%
Email: 7.92%
Dev: 0.04%


In [57]:
df['category'].value_counts()


,count
category,
Other,317076
CDN,34976
Dev,32820
Email,31930
